In [4]:
#-------------------------------------------------------------------------------
# elftools example: dwarf_lineprogram_filenames.py
#
# In the .debug_line section, the Dwarf line program generates a matrix
# of address-source references. This example demonstrates accessing the state
# of each line program entry to retrieve the underlying filenames.
#
# William Woodruff (william@yossarian.net)
# This code is in the public domain
#-------------------------------------------------------------------------------
from __future__ import print_function
from collections import defaultdict
import os
import sys
import posixpath

# If pyelftools is not installed, the example can also run from the root or
# examples/ dir of the source distribution.
# sys.path[0:0] = ['.', '..']

from elftools.elf.elffile import ELFFile


def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        dwarfinfo = elffile.get_dwarf_info()
        
        with open('lineprogram.s', 'w') as outFile:
            
            cc = 0
            for CU in dwarfinfo.iter_CUs():
                print('  Found a compile unit at offset %s, length %s' % (
                    CU.cu_offset, CU['unit_length']))

                # Every compilation unit in the DWARF information may or may not
                # have a corresponding line program in .debug_line.
                line_program = dwarfinfo.line_program_for_CU(CU)
                if line_program is None:
                    print('  DWARF info is missing a line program for this CU')
                    continue

                
                
                # Print a reverse mapping of filename -> #entries
#                 filename_map= line_entry_mapping(line_program)
#                 if(len(list(filename_map.items())))>1:
#                     print(list(filename_map.items()))

                CU_DIR_PATH = None
                CU_FILENAME = None
                for attr in CU.get_top_DIE().attributes.values():
                    if attr.name == 'DW_AT_name':
                        print(attr.value)
                        CU_FILENAME = attr.value
                    if attr.name == 'DW_AT_comp_dir':
                        print(attr.value)
                        CU_DIR_PATH = attr.value

                
                for le in line_program.get_entries():
                    cc+=1
                    
                    
                    if le.state is not None:
                         
                            src_filename = lpe_filename (line_program, le.state.file)

                            print(str(hex(le.state.address))+"  "+str(le.state.line)+"  "+    str(le.state.column)+"   dir:"+str(CU_DIR_PATH)+" file name"+str(CU_FILENAME)+"   "+ src_filename   +"\n")

def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename (line_program, lpe.state.file)
        filename_map[filename] += 1

#     for filename, lpe_count in filename_map.items():
#         print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename( line_program, file_index,debug= False):
    
    COMPILER_SUBSTRACT = 1
    
    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
    if debug:
        print(file_entries,'\n\n')
        print(lp_header)
    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index -COMPILER_SUBSTRACT]
    dir_index = file_entry["dir_index"]
#     print(file_entry , dir_index)

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()
    
    print(lp_header["include_directory"] , dir_index )
    directory = lp_header["include_directory"][dir_index -COMPILER_SUBSTRACT]
    return posixpath.join(directory, file_entry.name).decode()



# filePath = './../../binaries/c_many/stacktest'
filePath =  './../../binaries/cat'
process_file(filePath)


Processing file: ./../../binaries/cat
  Found a compile unit at offset 0, length 42
817
[b'src', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'/usr/include/x86_64-linux-gnu/sys', b'./lib'] 1
0x32f0  131  1   dir:817 file nameNone   src/cat.c

[b'src', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'/usr/include/x86_64-linux-gnu/sys', b'./lib'] 1
0x32f0  132  3   dir:817 file nameNone   src/cat.c

[b'src', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'/usr/include/x86_64-linux-gnu/sys', b'./lib'] 1
0x32f0  139  15   dir:817 file nameNone   src/cat.c

[b'src', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'

[b'lib', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'./lib'] 1
0x39c0  207  1   dir:817 file nameNone   lib/quotearg.c

[b'lib', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'./lib'] 1
0x39c0  233  3   dir:817 file nameNone   lib/quotearg.c

[b'lib', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'./lib'] 1
0x39c0  207  1   dir:817 file nameNone   lib/quotearg.c

[b'lib', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/include/x86_64-linux-gnu/bits/types', b'/usr/include', b'./lib'] 1
0x39cb  233  17   dir:817 file nameNone   lib/quotearg.c

[b'lib', b'/usr/include/x86_64-linux-gnu/bits', b'/usr/lib/gcc/x86_64-linux-gnu/11/include', b'/usr/inc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

